## Factory POS Clean-up and Data Extrapolation

In [ ]:
# Re-adjusting the code to fit the data structure
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

In [3]:
# Re-reading the CSV file
data = pd.read_csv('Factory POS.csv')

In [4]:
# Rename columns for clarity based on the file's first row which contains the actual column names
data.columns = ['Need State', 'Fiscal Week', '2021', '2022', '2023']

In [5]:
# Dropping the first row since it's now redundant
data = data.iloc[1:]

In [12]:
data.tail(10)

,Need State,Fiscal Week,2021,2022,2023
303,Need State 6,43,526055.25,485091.00,0.0
304,Need State 6,44,432303.75,409923.75,0.0
305,Need State 6,45,475594.50,448538.25,0.0
306,Need State 6,46,402365.25,359235.75,0.0
307,Need State 6,47,457254.75,524727.75,0.0
308,Need State 6,48,482283.00,585123.75,0.0
309,Need State 6,49,411926.25,540969.00,0.0
310,Need State 6,50,428814.75,584413.50,0.0
311,Need State 6,51,470064.75,581396.25,0.0
312,Need State 6,52,373771.50,552392.25,0.0


In [13]:
# Filter out 'Need State 6' directly within the original DataFrame
data = data[data['Need State'] != 'Need State 6'].reset_index(drop=True)

In [15]:
data.head(10)

,Need State,Fiscal Week,2021,2022,2023
0,Need State 1,1,317435.25,348637.50,372921.75
1,Need State 1,2,309234.75,343799.25,357329.25
2,Need State 1,3,331149.00,346592.25,352571.25
3,Need State 1,4,291885.00,344795.25,331310.25
4,Need State 1,5,308238.00,338146.50,383741.25
5,Need State 1,6,267614.25,307596.75,307953.75
6,Need State 1,7,277594.50,302883.00,318695.25
7,Need State 1,8,288708.00,343384.50,324699.75
8,Need State 1,9,305426.25,341754.00,316498.50
9,Need State 1,10,366468.75,396704.25,338994.75


In [16]:
#download intermediary dataframe as csv
# Specify the output file name and path
output_file_name = 'Intermediary_Step.csv'

# Use the to_csv method to save the DataFrame as a CSV
data.to_csv(output_file_name, index=False)

print(f"Data saved as '{output_file_name}'.")


Data saved as 'Intermediary_Step.csv'.


In [22]:
#Long format application for Factory POS data
melted_data = pd.melt(data, id_vars=[col for col in data.columns if col not in ['2021', '2022', '2023']], 
                      var_name='Year', value_name='Factory POS')

In [27]:
melted_data.head(15)

,Need State,Fiscal Week,Year,Factory POS
0,Need State 1,1,2021,317435.25
1,Need State 1,2,2021,309234.75
2,Need State 1,3,2021,331149.00
3,Need State 1,4,2021,291885.00
4,Need State 1,5,2021,308238.00
5,Need State 1,6,2021,267614.25
6,Need State 1,7,2021,277594.50
7,Need State 1,8,2021,288708.00
8,Need State 1,9,2021,305426.25
9,Need State 1,10,2021,366468.75


In [57]:
# projecting values for 2023 weeks 42-52 based on the 2021-2022 trend

# Convert 'Year' to int if necessary
melted_data['Year'] = melted_data['Year'].astype(int)

# Filter data to include only 2021 and 2022 for training
train_data = melted_data[melted_data['Year'].isin([2021, 2022])]

# Placeholder for predictions
predictions = []

# For each Need State, fit a model and predict weeks 42-52 for 2023
for need_state in train_data['Need State'].unique():
    ns_data = train_data[train_data['Need State'] == need_state]
    
    # Assuming 'Week' is a separate column and 'Factory POS' is the target
    X = ns_data[['Fiscal Week', 'Year']]
    y = ns_data['Factory POS']
    
    # Train a linear regression model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict for 2023, weeks 42-52
    for week in range(42, 53):
        predicted_pos = model.predict(np.array([[week, 2023]]))
        predictions.append({
            'Need State': need_state,
            'Year': 2023,
            'Fiscal Week': week,
            'Factory POS': predicted_pos[0]
        })

# Convert predictions list to a DataFrame
predictions_df = pd.DataFrame(predictions)      

/tmp/ipykernel_53/1505444576.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_data['Year'] = melted_data['Year'].astype(int)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not

In [58]:
predictions_df.head(20)

,Need State,Year,Fiscal Week,Factory POS
0,Need State 1,2023,42,540081.669016
1,Need State 1,2023,43,539340.406968
2,Need State 1,2023,44,538599.144919
3,Need State 1,2023,45,537857.882871
4,Need State 1,2023,46,537116.620822
5,Need State 1,2023,47,536375.358773
6,Need State 1,2023,48,535634.096725
7,Need State 1,2023,49,534892.834676
8,Need State 1,2023,50,534151.572628
9,Need State 1,2023,51,533410.310579


In [59]:
#Make the final data frame with the predicted values inserted

# Convert 'Week' and 'Year' to integers to avoid type comparison errors
melted_data['Fiscal Week'] = pd.to_numeric(melted_data['Fiscal Week'], errors='coerce').astype(int)
melted_data['Year'] = pd.to_numeric(melted_data['Year'], errors='coerce').astype(int)

# Now proceed with filtering and updating the DataFrame
# Remove existing entries for 2023 weeks 42-52 in melted_data to avoid duplicate entries
melted_data = melted_data[~((melted_data['Year'] == 2023) & (melted_data['Fiscal Week'].between(42, 52)))]

# Assuming predictions_df is prepared with the 'Week' column also as integer
# Merge predictions_df into melted_data
final_dataframe = pd.concat([melted_data, predictions_df], ignore_index=True)

# Sort final_dataframe by 'Need State', 'Year', and 'Week' for readability
final_dataframe.sort_values(by=['Need State', 'Year', 'Fiscal Week'], inplace=True)

/tmp/ipykernel_53/2663520742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_data['Fiscal Week'] = pd.to_numeric(melted_data['Fiscal Week'], errors='coerce').astype(int)
/tmp/ipykernel_53/2663520742.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_data['Year'] = pd.to_numeric(melted_data['Year'], errors='coerce').astype(int)


In [60]:
final_dataframe.tail(20)

,Need State,Fiscal Week,Year,Factory POS
716,Need State 5,33,2023,203193.000000
717,Need State 5,34,2023,121110.000000
718,Need State 5,35,2023,102096.750000
719,Need State 5,36,2023,131212.500000
720,Need State 5,37,2023,54834.750000
721,Need State 5,38,2023,49947.750000
722,Need State 5,39,2023,43021.500000
723,Need State 5,40,2023,57792.750000
724,Need State 5,41,2023,36111.000000
769,Need State 5,42,2023,113403.795174


In [63]:
output_file_name = 'Final_DataFrame.csv'

# Use the to_csv method to save the DataFrame as a CSV
final_dataframe.to_csv(output_file_name, index=False)

print(f"Data saved as '{output_file_name}'.")

Data saved as 'Final_DataFrame.csv'.
